In [14]:
import numpy as np
import pandas as pd
#import pandas_ta as ta

load data

In [16]:
filename = 'Macarons_data.xlsx'
data = pd.read_excel(filename)
data.columns = data.iloc[2,:].values
data = data.iloc[2:,]

data.set_index('timestamp', inplace=True)
df = data.loc[:,['MacaronsMidPrice','transportFees','exportTariff','importTariff','sugarPrice','sunlightIndex']]
df

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

features

In [4]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [ ]:
features = pd.DataFrame()

# Technical Indicators
window = 10  
for col in df.columns:
    features[f'{col}_rsi'] = ta.rsi(df[col], length=window)  # RSI
    features[f'{col}_roc'] = ta.roc(df[col], length=window)  # Rate of Change
    features[f'{col}_mom'] = ta.mom(df[col], length=window)  # Momentum
    features[f'{col}_ema50'] = ta.ema(df[col], length=50)  # EMA 50
    features[f'{col}_ema200'] = ta.ema(df[col], length=200)  # EMA 200
    features[f'{col}_zscore'] = ta.zscore(df[col], length=30)  # Z-Score
    features[f'{col}_mad'] = ta.mad(df[col], length=window)  # Mean Absolute Deviation
    features[f'{col}_kurtosis'] = ta.kurtosis(df[col], length=window)  # Kurtosis
    features[f'{col}_skew'] = ta.skew(df[col], length=window)  # Skewness

# Spread Features
df["export_import_spread"] = df["exportTariff"] - df["importTariff"]
df["sugar_transport_spread"] = df["sugarPrice"] - df["transportFees"]
df["sunlight_sugar_spread"] = df["sunlightIndex"] - df["sugarPrice"]

# Temporal Differences
for col in ["transportFees", "exportTariff", "importTariff", "sugarPrice", "sunlightIndex"]:
    df[f"{col}_diff"] = df[col].diff()

# Interaction Terms
df["sunlight_sugar_interaction"] = df["sunlightIndex_diff"] * df["sugarPrice_diff"]
df["transport_export_interaction"] = df["transportFees_diff"] * df["exportTariff_diff"]

# Momentum Indicators: Rolling Means (7-day, 30-day)
for col in df.columns:
    if col.endswith("_diff"):
        df[f"{col}_ma7"] = df[col].rolling(window=7).mean()
        df[f"{col}_ma30"] = df[col].rolling(window=30).mean()

# Volatility: Rolling Std Deviation (7-day, 30-day)
for col in ["transportFees", "exportTariff", "importTariff", "sugarPrice", "sunlightIndex"]:
    df[f"{col}_volatility_7d"] = df[col].rolling(window=7).std()
    df[f"{col}_volatility_30d"] = df[col].rolling(window=30).std()

# Non-linear Effects: Squared Differences
for col in df.columns:
    if col.endswith("_diff"):
        df[f"{col}_squared"] = df[col] ** 2

# Signal Ratios
df["sugar_transport_ratio"] = df["sugarPrice"] / df["transportFees"].replace(0, np.nan)
df["sunlight_sugar_ratio"] = df["sunlightIndex"] / df["sugarPrice"].replace(0, np.nan)
df["export_import_ratio"] = df["exportTariff"] / df["importTariff"].replace(0, np.nan)
